In [22]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [23]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [38]:

# Ruta del archivo CSV
archivo_csv = '../datos e2/clientes.csv'

# Configuración de la conexión
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    # Crear un cursor
    cursor = conexion.cursor()

    # Dropear la tabla si ya existe
    drop_table_query = "DROP TABLE IF EXISTS clientes;"
    cursor.execute(drop_table_query)
    print("Tabla clientes eliminada si existía")

    # Crear la tabla (ajusta esto según tu esquema de datos)
    crear_tabla_query = """
    CREATE TABLE IF NOT EXISTS clientes (
        id SERIAL,
        nombre varchar   NOT NULL,
        email varchar   NOT NULL,
        telefono varchar   NOT NULL,
        clave varchar       NOT NULL,
        direccion varchar   NOT NULL,
        comuna varchar    NOT NULL,
        CONSTRAINT pk_Cliente PRIMARY KEY (
            id))
    """
    cursor.execute(crear_tabla_query)
    conexion.commit()

    # Preparar el query de inserción
    insercion_query = """
    INSERT INTO clientes (nombre, email, telefono, clave, direccion, comuna)
    VALUES (%s, %s, %s, %s, %s, %s);
    """

    # Leer y procesar el archivo CSV
    with open(archivo_csv, 'r') as file:
        # Leer la primera línea para obtener los nombres de las columnas
        columnas = file.readline().strip().split(';')
        lista_clientes = []
        cliente_aux = ""
        # Iterar sobre cada línea en el archivo CSV
        for linea in file:
            # Dividir la línea en valores
            valores = linea.strip().split(';')
            if len(valores) != 6 :
                if len(cliente_aux.strip().split(';')) == 6 or len(cliente_aux.strip().split(';')) == 0:
                    cliente_aux = linea
                else:
                    cliente_aux += linea
                valores = cliente_aux
            else:
                lista_clientes.append(valores)
        for cliente in lista_clientes:
            try:
                cursor.execute(insercion_query, (cliente[0], cliente[1], cliente[2], cliente[3], cliente[4], cliente[5]))
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

    # Confirmar la inserción
    conexion.commit()

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

<connection object at 0x000001CD34952AC0; dsn: 'user=grupo64 password=xxx dbname=grupo64e2 host=pavlov.ing.puc.cl port=5432', closed: 0>
<cursor object at 0x000001CD34EAF220; closed: 0>
Tabla clientes eliminada si existía
['cliente', 'email', 'telefono', 'clave', 'direccion', 'comuna']
